In [1]:
from pprint import pprint

In [2]:
from data_load import load_mit_db
from utils import PrintTime

In [3]:
db_params = {
    "DS": None,
    "is_reduce": True,
    "do_preprocess":True,
    "ws": [90, 90],
}

ml_features_params={
    "maxRR": True,
    "use_RR": False,
    "norm_RR": True,
    "leads_flag": [0, 1],
    "compute_morph": ['resample_10', 'lbp', 'hbf5', 'wvlt', 'HOS'],
    "DS": "DS1",  # for wvlt+pca
    "ws": [90, 90],  # for mymorph
}


db_trn_params = {}
db_trn_params.update(db_params)
db_trn_params["DS"] = "DS1"
ml_trn_features_params = ml_features_params
ml_trn_features_params = {}
ml_trn_features_params.update(ml_features_params)
ml_trn_features_params["DS"] = "DS1"
print(db_trn_params)

db_val_params = {}
db_val_params.update(db_params)
db_val_params["DS"] = "DS2"
ml_val_features_params = ml_features_params
ml_val_features_params = {}
ml_val_features_params.update(ml_features_params)
ml_val_features_params["DS"] = "DS2"
print(db_val_params)

load_trn_params = {}
load_trn_params.update(db_trn_params)
load_trn_params.update(ml_trn_features_params)
load_trn_params.pop("leads_flag")

load_val_params = {}
load_val_params.update(db_val_params)
load_val_params.update(ml_val_features_params)
load_val_params.pop("leads_flag")

{'DS': 'DS1', 'is_reduce': True, 'do_preprocess': True, 'ws': [90, 90]}
{'DS': 'DS2', 'is_reduce': True, 'do_preprocess': True, 'ws': [90, 90]}


[0, 1]

In [4]:
pprint(load_trn_params)
pprint(load_val_params)

{'DS': 'DS1',
 'compute_morph': ['resample_10', 'lbp', 'hbf5', 'wvlt', 'HOS'],
 'do_preprocess': True,
 'is_reduce': True,
 'maxRR': True,
 'norm_RR': True,
 'use_RR': False,
 'ws': [90, 90]}
{'DS': 'DS2',
 'compute_morph': ['resample_10', 'lbp', 'hbf5', 'wvlt', 'HOS'],
 'do_preprocess': True,
 'is_reduce': True,
 'maxRR': True,
 'norm_RR': True,
 'use_RR': False,
 'ws': [90, 90]}


In [5]:
tr_features, tr_labels, tr_patient_num_beats = load_mit_db(**load_trn_params)

Loading pickle: /home/congyu/dataset/ECG/mitdb/ml_learning/features/w_90_90_DS1_rm_bsline_maxRR_norm_RR_resample_10_lbp_hbf5_wvlt_HOS_reduced_MLII_V1.pkl...


In [6]:
print(tr_features.shape)
print(tr_labels.shape)

(42617, 152)
(42617,)


In [7]:
eval_features, eval_labels, eval_patient_num_beats = load_mit_db(**load_val_params)

Loading pickle: /home/congyu/dataset/ECG/mitdb/ml_learning/features/w_90_90_DS2_rm_bsline_maxRR_norm_RR_resample_10_lbp_hbf5_wvlt_HOS_reduced_MLII_V1.pkl...


In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()
scaler.fit(tr_features)
tr_features_scaled = scaler.transform(tr_features)
eval_features_scaled = scaler.transform(eval_features)

In [10]:
import numpy as np

In [11]:
from sklearn import svm
import time

In [12]:
model_kwargs = {}
verbose = False

C_value = model_kwargs.get("c_value", 1)
gamma_value = model_kwargs.get("gamma_value", 0)
multi_mode = model_kwargs.get("multi_model", "ovr")

# TODO load best params from cross validation!
use_probability = False

class_weights = {}
for c in range(4):
    class_weights.update({c: len(tr_labels) / float(np.count_nonzero(tr_labels == c))})

# class_weight='balanced',
if gamma_value != 0.0:  # NOTE 0.0 means 1/n_features default value
    svm_model = svm.SVC(C=C_value, kernel='rbf', degree=3, gamma=gamma_value,
                        coef0=0.0, shrinking=True, probability=use_probability, tol=0.001,
                        cache_size=200, class_weight=class_weights, verbose=verbose,
                        max_iter=-1, decision_function_shape=multi_mode, random_state=None)
else:
    svm_model = svm.SVC(C=C_value, kernel='rbf', degree=3, gamma='auto',
                        coef0=0.0, shrinking=True, probability=use_probability, tol=0.001,
                        cache_size=200, class_weight=class_weights, verbose=verbose,
                        max_iter=-1, decision_function_shape=multi_mode, random_state=None)



In [13]:
with PrintTime("train a model"):
    svm_model.fit(tr_features_scaled, tr_labels)


working on train a model ...
time for train a model: 40.93 sec



In [14]:
y_pred = svm_model.predict(eval_features_scaled)

In [15]:
y_pred.shape

(42285,)

In [16]:
from collections import Counter

In [18]:
c = Counter(y_pred[0:10000])

In [19]:
print(c)

Counter({1: 8960, 3: 713, 2: 180, 0: 147})


In [21]:
from sklearn.metrics import f1_score, classification_report
from config import AAMI_CLASSES

In [22]:
print("marco f1 score", f1_score(eval_labels, y_pred, average='macro'))

marco f1 score 0.4332208101606776


In [23]:
print(classification_report(eval_labels, y_pred, target_names=AAMI_CLASSES))

              precision    recall  f1-score   support

           F       0.00      0.00      0.00       388
           N       0.95      0.88      0.91     36667
        SVEB       0.36      0.13      0.19      2014
         VEB       0.47      0.96      0.63      3216

    accuracy                           0.84     42285
   macro avg       0.44      0.49      0.43     42285
weighted avg       0.87      0.84      0.85     42285

